## Taller ML Flow
Ml flow es una librería de código abierto que permite gestionar el ciclo de vida de los modelos de machine learning. En este taller vamos a ver cómo se puede utilizar para gestionar los modelos de machine learning.

Primero creemos un ambiente de usando 
```bash
python -m venv mlflow
.\mlflow\bin\activate
```

instalamos mlflow
```bash
pip install mlflow
$Env:MLFLOW_TRACKING_URI="http://localhost:5000"
```

puede tardar un poco en instalar, pero una vez instalado podemos correr el siguiente comando para ver si se instaló correctamente
```bash
mlflow --version
```

para activar la interfaz web de mlflow corremos el siguiente comando
```bash
mlflow ui
```
y abrimos el navegador en la dirección que nos indica el comando probablemente sea http://127.0.0.1:5000

Ahora podemos empezar a trabajar con mlflow, las celdas de abajo muestran cómo registrar variables 

In [14]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("check-localhost-connection")

with mlflow.start_run():
    mlflow.log_metric("foo", 1)
    mlflow.log_metric("bar", 2)

2024/11/11 14:15:21 INFO mlflow.tracking._tracking_service.client: 🏃 View run youthful-goose-994 at: http://localhost:5000/#/experiments/197888178222126090/runs/25864edee3bd402fb4d60246952f3c77.
2024/11/11 14:15:21 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/197888178222126090.


Abra un powershell como administrador y corra el siguiente comando
```bash

Invoke-WebRequest -UseBasicParsing -Uri "https://raw.githubusercontent.com/pyenv-win/pyenv-win/master/pyenv-win/install-pyenv-win.ps1" -OutFile "./install-pyenv-win.ps1"; &"./install-pyenv-win.ps1"
.\pyenv-win install 3.12.0
```

```bash
New-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Control\FileSystem" -Name "LongPathsEnabled" -Value 1 -PropertyType DWORD -Force
```


Ahora probemos con un modelo real. 

Primero instalemos keras y tensorflow 

Abrimos un terminal y corremos los siguientes comandos
```bash
.\mlflow\bin\activate
pip install tensorflow
pip install keras
python -m ensurepip --upgrade
python -m pip install --upgrade setuptools
python -m distutils.core --help
```
Estos comandos instalan tensorflow y keras, y actualizan setuptools y distutils.
Debemos ahora reiniciar el kernel de jupyter para que los cambios surtan efecto.

Solo falta un paquete extra llamado hyperopt que nos va a permitir hacer optimización de hiperparámetros
```bash
pip install hyperopt
```

El ejemplo que se encuentra en este notebook es una adaptación de un ejemplo de la documentación de mlflow:

https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

import mlflow
from mlflow.models import infer_signature

In [2]:
# Cargamos un dataset clásico de sklearn para regresión
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)


In [4]:

def train_model_tree(params, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture: Decision Tree Regressor
    # Our model is a simple decision tree regressor using the parameters provided
    model = DecisionTreeRegressor(
        max_depth=params.get("max_depth", None),
        min_samples_split=params.get("min_samples_split", 2),
        min_samples_leaf=params.get("min_samples_leaf", 1)
    )

    # Train the model
    model.fit(train_x, train_y)

    # Evaluate the model
    valid_predictions = model.predict(valid_x)
    eval_rmse = np.sqrt(mean_squared_error(valid_y, valid_predictions))

    # Log parameters and results
    with mlflow.start_run(nested=True):
        # This step is crucial to compare models and understand how they behave in different scenarios
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.sklearn.log_model(model, "model")

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [6]:
def objective_tree(params):
    # MLflow will track the parameters and results for each run
    result = train_model_tree(
        params,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [7]:
space_tree= {
    "max_depth": hp.choice("max_depth", range(1, 20)),
    "min_samples_split": hp.choice("min_samples_split", range(2, 20)),
    "min_samples_leaf": hp.choice("min_samples_leaf", range(1, 20)),
}

In [9]:
mlflow.set_experiment("/wine-quality-tree")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective_tree,
        space=space_tree,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.sklearn.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

  0%|          | 0/8 [00:00<?, ?trial/s, best loss=?]

2024/11/14 12:30:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 12%|█▎        | 1/8 [00:04<00:32,  4.67s/trial, best loss: 0.8105583442615206]

2024/11/14 12:30:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 25%|██▌       | 2/8 [00:08<00:26,  4.35s/trial, best loss: 0.8105583442615206]

...

 75%|███████▌  | 6/8 [00:24<00:08,  4.10s/trial, best loss: 0.7664869271900654]

2024/11/14 12:31:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 88%|████████▊ | 7/8 [00:28<00:04,  4.05s/trial, best loss: 0.7664869271900654]

2024/11/14 12:31:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 8/8 [00:32<00:00,  4.10s/trial, best loss: 0.7539281965810375]
Best parameters: {'max_depth': np.int64(5), 'min_samples_leaf': np.int64(15), 'min_samples_split': np.int64(14)}
Best eval rmse: 0.7539281965810375


Ahora hagamos el mismo ejercicio con una red neuronal

In [ ]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    # Nuestro modelo es una red neuronal secuencial con una capa densa de 64 neuronas y una capa de salida de una neurona
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    # Los hiperparametros son el learning rate y el momentum del optimizador SGD
    # Esto permite que el modelo ajuste sus pesos basándose en la dirección 
    # y magnitud del gradiente de la función de pérdida.
    
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        # Este paso es importante para poder comparar modelos
        # y entender cómo se comportan en distintos escenarios
        
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [11]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result


In [ ]:
# Acá se define el espacio de búsqueda de hiperparámetros
# lr: desde 1e-5 hasta 1e-1 en escala logarítmica 
# momentum: desde 0 hasta 1
# es como si pasaramos tuplas de (lr, momentum) = (1e-5, 0), (1e-5, 0.1), (1e-5, 0.2), ... (1e-1, 0.9), (1e-1, 1)
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}


In [ ]:
mlflow.set_experiment("/wine-quality-nn")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2024/11/11 13:53:34 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 32s 722ms/step - loss: 34.7678 - root_mean_squared_error: 5.9561
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 35.6320 - root_mean_squared_error: 5.9690
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 35.4793 - root_mean_squared_error: 5.9690
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 35.7497 - root_mean_squared_error: 5.9789
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 35.7710 - root_mean_squared_error: 5.9807
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 35.8156 - root_mean_squared_error: 5.9845   
 9/46 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 35.7020 - root_mean_squared_error: 5.9750
10/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 35.4906 - root_mean_squared_error: 5.9572
11/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 35.5425 - root_mean_squared_error: 5.9572
12/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 35.3505 - root_mean_squared_error: 5.9454
13/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3

Ahora vaya a http://127.0.0.1:5000 y vea cómo se registran los modelos y los experimentos en la interfaz web de mlflow

Acceda al modelo que se registró en la interfaz web y vea las métricas y los hiperparámetros que se registraron.

Elija un modelo y copie el id del modelo, luego corra la celda de abajo con el id del modelo que eligió para ver cómo se puede cargar un modelo de mlflow

In [ ]:
run_id = "680bbef7b6e44bd9a8df6201e99ac602" # remplace por un run_id válido para su caso
model = mlflow.tensorflow.load_model(f"runs:/{run_id}/model")
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_4 (Normalization) │ (None, 11)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,668 (6.52 KB)

 Trainable params: 833 (3.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 835 (3.27 KB)

Vaya y registre el modelo como en la siguiente imagen

![instruccion](imgs/instruccion.png)



Para servir el modelo en una api puede correr el siguiente comando
```bash
mlflow models serve -m "models:/wine_predictor/1" --no-conda --port 5002
```

Puede hacer un llamado a la API de MLflow para obtener los resultados de la predicción de un modelo. Para ello, puede correr el siguiente código en un script en un terminal nuevo
```bash
Invoke-WebRequest -Uri "http://localhost:5002/invocations" `
    -Method POST `
    -Body '{"dataframe_split": {"columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"], "data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}' `
    -ContentType "application/json"
``` 